#1 Introducción


En el siguiente cuaderno obtenemos la matriz transpuesta de una matriz de números flotantes de manera secuencial utilizando solo procesador CPU.

El objetivo es comparar los tiempos de ejecución de ambas versiones (secuencial vs paralelismo) del algoritmo que obtiene la matriz transpuesta.

#2 Preparado del ambiente

Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 6.9MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621228 sha256=88411c14ef43a7315ebd9c0770f9c0ddc13cc6a2bcb4765f5a9338fc10e74b7d
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=a5949b691559d6616780389cda54447ced4480a53f22c9751002496f3f77a435
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3 Desarrollo

In [1]:
#@title Parametros { vertical-output: true, display-mode: "both" }
ordenMatriz =  5#@param {type:"number"}
try:
  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule
  import numpy

  # CPU - Defino la memoria de las matrices en cpu.
  matriz = numpy.random.random((ordenMatriz,ordenMatriz))
  matriz = matriz.astype(numpy.float32())

  matrizT = numpy.empty_like(matriz)

  # CPU - reservo la memoria GPU.
  matrizGpu = cuda.mem_alloc(matriz.nbytes)
  matriztGpu = cuda.mem_alloc(matrizT.nbytes)

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(matrizGpu,matriz)
  cuda.memcpy_htod(matriztGpu,matrizT)

  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void transponer(int n, float *matriz ,float *matrizT ,float *pvector1)
  {
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      int idy = threadIdx.y + blockIdx.y*blockDim.y;
      
      

      if(idx<n && idy<n)
      {
        matrizT[idy*n+idx] = matriz[idx*n+idy];
      }
  }

  """)

  kernel = module.get_function("transponer")

  dim_hilo_x = 16
  dim_bloque_x = numpy.int( (ordenMatriz+dim_hilo_x-1) / dim_hilo_x )

  dim_hilo_y = 16
  dim_bloque_y = numpy.int( (ordenMatriz+dim_hilo_y-1) / dim_hilo_y )

  print( "Thread x: ", dim_hilo_x, ", Bloque x:", dim_bloque_x )
  print( "Thread y: ", dim_hilo_y, ", Bloque y:", dim_bloque_y )

  kernel( numpy.int32(ordenMatriz), matrizGpu ,matriztGpu, block=( dim_hilo_x, dim_hilo_y, 1 ),grid=(dim_bloque_x, dim_bloque_y,1) )

  cuda.memcpy_dtoh(matrizT,matriztGpu)

  print("matriz original: ")
  print(matriz)
  print("")
  print("matriz transpuesta: ")
  print(matrizT)

except ModuleNotFoundError :
      print("No se compilo el código del armado del ambiente")
      print("Compile el armado del ambiente y vuelva a intentarlo")
except ValueError :
  print("se ingreso un orden (tamaño de la matriz) menor a 0")
  print("ingrese un orden mayor a 0")
except pycuda.driver.LogicError:
  print("Ha ingresado un orden de la matriz igual a 0 y CUDA no puede reservar una matriz de tamaño 0")
  print("ingrese un orden para la matriz mayor a 0")
      
 

No se compilo el código del armado del ambiente
Compile el armado del ambiente y vuelva a intentarlo


#4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño del vector desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( ordenMatriz ) | Inicializa las matrices
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel donde se transpone la matriz
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_hilo/dim_bloque   | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.

#5 Conclusiones

#6 Bibliografia

[1] MARKDOWN SYNTAX Colab:[PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [pagina colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)